In [1]:
import torch
import torch.nn as nn
import torch.nn.functional as F
from torch.nn.parameter import Parameter
import numpy as np
import pandas
import os
from tqdm import tqdm
import random

from graph import Graph
from knowledgegraph import KnowledgeGraph
from rule import Rule
from train import train
from train import train_step_by_step
from train import train_pow_decay
from device import device
import pickle

# data_path = 'data/wn18rr/train.txt'
data_path = 'data/umls/train.txt'
# data_path = 'temp.txt'
# data_path = 'data/data_2_rule.txt'
# data_path = 'data/family/train.txt'
# data_path = 'data/noisy.txt'
# data_path = 'data/kinship/train.txt'
# data_path = 'testdata.txt'
KG = KnowledgeGraph(data_path)

EPOCH = 100
BATCH_SIZE = 1000
rule_body_len = 3
target_predicate = 0

c:\Users\tourx\source\repos\sota\knowledgegraph.py:13: FutureWarning: The series.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  total_entity = data[0].append(data[2])


In [2]:
n_step = 2
target_predicate = KG.predicate2id["affects"]
pos_triplets = KG.graph.A.clone().detach()[target_predicate]
neg_triplets = (1.0 - KG.graph.A).detach()[target_predicate]

model = Rule(KG.graph.n_edge_type, n_step, False).to(device)
ans = model.forward(KG.graph, n_step)
l1 = torch.sum(ans * (neg_triplets + pos_triplets))
l0 = torch.sum(ans * pos_triplets)
loss = -l0 / l1 - l0 / (torch.sum(pos_triplets))
loss.requires_grad_(True)
loss.backward()

for i in range(model.n_step):
    print(model.R[i].R._grad)

tensor([-8.4020e-03,  2.6536e-03,  1.8966e-03, -1.5162e-02,  1.0090e-05,
         8.7561e-04,  1.1913e-03,  1.1885e-03,  9.3481e-05,  9.8126e-05,
        -3.0334e-04,  6.9087e-03, -1.6431e-04,  4.1816e-04, -2.5405e-03,
        -2.7186e-03,  2.2774e-04,  6.2399e-04,  1.9002e-04,  7.0704e-04,
         4.2491e-06,  1.1973e-03, -9.6914e-04,  1.7772e-04,  2.8229e-03,
        -1.4970e-03,  4.6269e-04,  9.8825e-05,  7.8879e-04,  4.6318e-05,
        -8.5570e-04,  1.6616e-03, -5.8131e-05,  2.2666e-04,  1.3312e-04,
         2.3289e-05,  4.4015e-05,  2.5911e-04,  5.4557e-05,  9.8869e-05,
        -9.8272e-05,  2.6052e-05,  2.2828e-05,  1.1313e-05,  2.2828e-05,
         2.6982e-03,  2.7044e-03, -3.3053e-05,  3.3545e-03,  3.2023e-04,
         1.9996e-03, -6.2058e-04,  3.2877e-04,  1.0847e-03,  2.8049e-03,
        -1.8713e-04, -7.2825e-05,  9.7920e-04,  1.0727e-04, -1.5869e-03,
        -5.2734e-03, -4.1503e-03, -8.7444e-06,  3.1476e-05, -5.9147e-05,
        -4.1101e-05,  3.4144e-03, -1.0642e-03,  5.3

In [11]:
l1 - l0

tensor(0.0076, device='cuda:0', grad_fn=<SubBackward0>)